## Data Preparation

Imports

In [0]:
import pandas as pd
import numpy as np

In [162]:
df_raw = pd.read_csv("influenza.csv")
df_raw.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Disease,Count,County Centroid,FIPS
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,INFLUENZA_UNSPECIFIED,0,"(43.7863965, -75.4426165)",36049
1,2014-2015,WESTERN,MONROE,53,01/03/2015,INFLUENZA_B,2,"(43.174966, -77.639663)",36055
2,2015-2016,METRO,NASSAU,52,01/02/2016,INFLUENZA_B,2,"(40.7506561, -73.582905)",36059
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,INFLUENZA_B,0,"(44.1089711, -73.7775726)",36031
4,2013-2014,METRO,ULSTER,41,10/12/2013,INFLUENZA_B,1,"(41.9472321, -74.265447)",36111
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,INFLUENZA_B,0,"(43.6578786, -74.5024556)",36041
6,2017-2018,METRO,NASSAU,49,12/09/2017,INFLUENZA_A,16,"(40.7506561, -73.582905)",36059
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,INFLUENZA_A,17,"(43.5551053, -73.8381388)",36113
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,INFLUENZA_A,7,"(42.2667252, -77.3855253)",36101
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,INFLUENZA_A,98,"(42.8175421, -74.0435834)",36093


Add year column to a new dataframe created as df_year

In [163]:
df_year = df_raw.copy()
df_year['year'] = df_raw['Week Ending Date']
df_year['year'] = df_year['year'].str.slice(start = 6)
df_year.head()

,Season,Region,County,CDC Week,Week Ending Date,Disease,Count,County Centroid,FIPS,year
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,INFLUENZA_UNSPECIFIED,0,"(43.7863965, -75.4426165)",36049,2016
1,2014-2015,WESTERN,MONROE,53,01/03/2015,INFLUENZA_B,2,"(43.174966, -77.639663)",36055,2015
2,2015-2016,METRO,NASSAU,52,01/02/2016,INFLUENZA_B,2,"(40.7506561, -73.582905)",36059,2016
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,INFLUENZA_B,0,"(44.1089711, -73.7775726)",36031,2019
4,2013-2014,METRO,ULSTER,41,10/12/2013,INFLUENZA_B,1,"(41.9472321, -74.265447)",36111,2013


Separating disease column to different Influenza types based on their count

In [164]:
#One hot encoding Disease column

df_count = pd.get_dummies(df_year, columns=['Disease'])
#df_count.shape
df_count.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Count,County Centroid,FIPS,year,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,0,"(43.7863965, -75.4426165)",36049,2016,0,0,1
1,2014-2015,WESTERN,MONROE,53,01/03/2015,2,"(43.174966, -77.639663)",36055,2015,0,1,0
2,2015-2016,METRO,NASSAU,52,01/02/2016,2,"(40.7506561, -73.582905)",36059,2016,0,1,0
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,0,"(44.1089711, -73.7775726)",36031,2019,0,1,0
4,2013-2014,METRO,ULSTER,41,10/12/2013,1,"(41.9472321, -74.265447)",36111,2013,0,1,0
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,0,"(43.6578786, -74.5024556)",36041,2016,0,1,0
6,2017-2018,METRO,NASSAU,49,12/09/2017,16,"(40.7506561, -73.582905)",36059,2017,1,0,0
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,17,"(43.5551053, -73.8381388)",36113,2018,1,0,0
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,7,"(42.2667252, -77.3855253)",36101,2017,1,0,0
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,98,"(42.8175421, -74.0435834)",36093,2019,1,0,0


In [165]:
#Multiplying disease counts with new diseases columns (A, B, Unspecified)

df_count[["Disease_INFLUENZA_A", "Disease_INFLUENZA_B", "Disease_INFLUENZA_UNSPECIFIED"]] = df_count[["Disease_INFLUENZA_A", "Disease_INFLUENZA_B", "Disease_INFLUENZA_UNSPECIFIED"]].multiply(df["Count"], axis="index")
df_count.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Count,County Centroid,FIPS,year,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,0,"(43.7863965, -75.4426165)",36049,2016,0,0,0
1,2014-2015,WESTERN,MONROE,53,01/03/2015,2,"(43.174966, -77.639663)",36055,2015,0,2,0
2,2015-2016,METRO,NASSAU,52,01/02/2016,2,"(40.7506561, -73.582905)",36059,2016,0,2,0
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,0,"(44.1089711, -73.7775726)",36031,2019,0,0,0
4,2013-2014,METRO,ULSTER,41,10/12/2013,1,"(41.9472321, -74.265447)",36111,2013,0,1,0
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,0,"(43.6578786, -74.5024556)",36041,2016,0,0,0
6,2017-2018,METRO,NASSAU,49,12/09/2017,16,"(40.7506561, -73.582905)",36059,2017,16,0,0
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,17,"(43.5551053, -73.8381388)",36113,2018,17,0,0
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,7,"(42.2667252, -77.3855253)",36101,2017,7,0,0
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,98,"(42.8175421, -74.0435834)",36093,2019,98,0,0


In [0]:
#df_count['Week'] = pd.to_datetime(df_count.year.astype(str), format='%Y') + \
#             pd.to_timedelta(df_count['CDC Week'].mul(7).astype(str) + ' days')

In [0]:
#df_count = df_count.sort_values(by = 'Week')

In [168]:
# Sum different Influenza types
df_sum = df_count.groupby(['County','FIPS','County Centroid','year','CDC Week','Week Ending Date'])["Count","Disease_INFLUENZA_A","Disease_INFLUENZA_B","Disease_INFLUENZA_UNSPECIFIED"].sum().reset_index()
df_sum.head(15)

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0
1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0
2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0
3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0
4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0
5,ALBANY,36001,"(42.5882713, -73.9740136)",2009,45,11/14/2009,61,61,0,0
6,ALBANY,36001,"(42.5882713, -73.9740136)",2009,46,11/21/2009,29,29,0,0
7,ALBANY,36001,"(42.5882713, -73.9740136)",2009,47,11/28/2009,17,17,0,0
8,ALBANY,36001,"(42.5882713, -73.9740136)",2009,48,12/05/2009,8,8,0,0
9,ALBANY,36001,"(42.5882713, -73.9740136)",2009,49,12/12/2009,5,5,0,0


In [0]:
df_sum.to_csv('df_sum.csv')

In [170]:
# Testing for above logic
df_sum.loc[(df_sum['County'] == 'KINGS') & (df_sum['Week Ending Date'] == '02/10/2018')]

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
7263,KINGS,36047,"(40.6782, -73.9442)",2018,6,02/10/2018,1404,919,473,12
